Load dataset. It contains translations from English to Spanish, so swap the order of the phrases. Also add `\t` and `\n` as the start and stop tokens in the target sequences. 

In [1]:
start_token = "\t"
stop_token = "\n"

with open("data/spa.txt", "r", encoding="utf-8") as f:
    samples = f.read().split("\n")

samples = [sample.strip().split("\t")
           for sample in samples if len(sample.strip()) > 0]

samples = [(es, start_token + en + stop_token)
           for en, es in samples if len(es) < 45]

In [2]:
len(samples)

99423

In [3]:
print(samples[:5])

[('Ve.', '\tGo.\n'), ('Vete.', '\tGo.\n'), ('Vaya.', '\tGo.\n'), ('Váyase.', '\tGo.\n'), ('Hola.', '\tHi.\n')]


Split data into train and validation sets.

In [4]:
from sklearn.model_selection import train_test_split

train_samples, valid_samples = train_test_split(samples, train_size=.8, random_state=42)

In [5]:
len(train_samples)

79538

In [6]:
len(valid_samples)

19885

Determine the training vocabulary. Those are the only tokens you can trust the model will know how to handle. 

In [7]:
in_vocab = set()
out_vocab = set()

for in_seq, out_seq in train_samples:
    in_vocab.update(in_seq)
    out_vocab.update(out_seq)
    
in_vocab_size = len(in_vocab)
out_vocab_size = len(out_vocab)
print("Input vocab size:", in_vocab_size)
print("Output vocab size:", out_vocab_size)

Input vocab size: 101
Output vocab size: 87


In [8]:
print(sorted(in_vocab))

[' ', '!', '"', '$', '%', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '°', 'º', '»', '¿', 'Á', 'É', 'Ó', 'Ú', 'á', 'è', 'é', 'í', 'ñ', 'ó', 'ö', 'ú', 'ü', 'ś', 'с', '—', '€']


In [9]:
print(sorted(out_vocab))

['\t', '\n', ' ', '!', '"', '$', '%', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'á', 'ã', 'è', 'é', 'ö', '‘', '’', '₂', '€']


Go through validation set and remove any tokens not present in the training set.

In [10]:
tmp_samples = []
for in_seq, out_seq in valid_samples:
    tmp_in_seq = [c for c in in_seq if c in in_vocab]
    tmp_out_seq = [c for c in out_seq if c in out_vocab]

    tmp_samples.append(("".join(tmp_in_seq), "".join(tmp_out_seq)))
    
valid_samples = tmp_samples

Build sequence-to-sequence model.

In [11]:
import keras
from keras.layers import Dense, Input, LSTM, Masking
from keras.models import Model

In [12]:
latent_dim = 256

encoder_in = Input(shape=(None, in_vocab_size), name="encoder_in")
encoder_mask = Masking(name="encoder_mask")(encoder_in)
encoder_lstm = LSTM(latent_dim, return_state=True, recurrent_dropout=0.3, name="encoder_lstm")
_, encoder_h, encoder_c = encoder_lstm(encoder_mask)

2021-12-23 21:41:23.682200: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
decoder_in = Input(shape=(None, out_vocab_size), name="decoder_in")

decoder_mask = Masking(name="decoder_mask")(decoder_in)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout=0.2, recurrent_dropout=0.3, name="decoder_lstm")
decoder_lstm_out, _, _ = decoder_lstm(decoder_mask, initial_state=[encoder_h, encoder_c])
decoder_dense = Dense(out_vocab_size, activation="softmax", name="decoder_out")
decoder_out = decoder_dense(decoder_lstm_out)

In [14]:
seq2seq_model = Model([encoder_in, decoder_in], decoder_out)
seq2seq_model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [15]:
seq2seq_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_in (InputLayer)        [(None, None, 101)]  0           []                               
                                                                                                  
 decoder_in (InputLayer)        [(None, None, 87)]   0           []                               
                                                                                                  
 encoder_mask (Masking)         (None, None, 101)    0           ['encoder_in[0][0]']             
                                                                                                  
 decoder_mask (Masking)         (None, None, 87)     0           ['decoder_in[0][0]']             
                                                                                              

Create maps to convert characters to and from ints. 

In [16]:
in_token2int = {token : i for i, token in enumerate(sorted(in_vocab))}
out_token2int = {token : i for i, token in enumerate(sorted(out_vocab))}
out_int2token = {i : token for (token, i) in out_token2int.items()}

In [17]:
import json

with open("esCharToInt.json", "w") as f:
    json.dump(in_token2int, f)
with open("intToEnChar.json", "w") as f:
    json.dump(out_int2token, f)

Create helper functions for one-hot encoding sequences for use with the model.

In [19]:
import numpy as np

def make_batch_storage(batch_size, in_seq_len, out_seq_len):
    
    enc_in_seqs = np.zeros(
        (batch_size, in_seq_len, in_vocab_size),
        dtype=np.float32)

    dec_in_seqs = np.zeros(
        (batch_size, out_seq_len, out_vocab_size),
        dtype=np.float32)

    dec_out_seqs = np.zeros(
        (batch_size, out_seq_len, out_vocab_size),
        dtype=np.float32)
        
    return enc_in_seqs, dec_in_seqs, dec_out_seqs

In [20]:
def encode_batch(samples):
    batch_size = len(samples)
    max_in_length = max([len(seq) for seq, _ in samples])
    max_out_length = max([len(seq) for _, seq in samples])

    enc_in_seqs, dec_in_seqs, dec_out_seqs = make_batch_storage(
        batch_size, max_in_length, max_out_length)
    
    for i, (in_seq, out_seq) in enumerate(samples):
        for time_step, token in enumerate(in_seq):
            enc_in_seqs[i, time_step, in_token2int[token]] = 1

        for time_step, token in enumerate(out_seq):
            dec_in_seqs[i, time_step, out_token2int[token]] = 1

        for time_step, token in enumerate(out_seq[1:]):
            dec_out_seqs[i, time_step, out_token2int[token]] = 1
            
    return enc_in_seqs, dec_in_seqs, dec_out_seqs

Train model

In [21]:
from seq2seq_util import Seq2SeqBatchGenerator

batch_size = 64
train_generator = Seq2SeqBatchGenerator(train_samples, batch_size, encode_batch)
valid_generator = Seq2SeqBatchGenerator(valid_samples, batch_size, encode_batch)

In [22]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

seq2seq_model.fit_generator(train_generator, epochs=500,
                            validation_data=valid_generator,
                            callbacks=[early_stopping])

<ipython-input-22-57ff8f760a77>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  seq2seq_model.fit_generator(train_generator, epochs=500,


Epoch 1/500
1247/1247 [==============================] - 420s 333ms/step - loss: 1.4484 - val_loss: 1.1514
Epoch 2/500
1247/1247 [==============================] - 417s 335ms/step - loss: 1.1478 - val_loss: 0.9840
Epoch 3/500
1247/1247 [==============================] - 409s 328ms/step - loss: 1.0373 - val_loss: 0.9091
Epoch 4/500
1247/1247 [==============================] - 407s 326ms/step - loss: 0.9852 - val_loss: 0.8948
Epoch 5/500
1247/1247 [==============================] - 413s 331ms/step - loss: 0.9515 - val_loss: 0.8372
Epoch 6/500
1247/1247 [==============================] - 408s 327ms/step - loss: 0.9288 - val_loss: 0.8081
Epoch 7/500
1247/1247 [==============================] - 407s 326ms/step - loss: 0.9114 - val_loss: 0.7972
Epoch 8/500
1247/1247 [==============================] - 408s 327ms/step - loss: 0.9007 - val_loss: 0.7865
Epoch 9/500
1247/1247 [==============================] - 408s 327ms/step - loss: 0.8926 - val_loss: 0.7737
Epoch 10/500
1247/1247 [=============

Create encoder/decoder models for inference

In [23]:
inf_encoder = Model(encoder_in, [encoder_h, encoder_c])

In [24]:
inf_encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_in (InputLayer)     [(None, None, 101)]       0         
                                                                 
 encoder_mask (Masking)      (None, None, 101)         0         
                                                                 
 encoder_lstm (LSTM)         [(None, 256),             366592    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 366,592
Trainable params: 366,592
Non-trainable params: 0
_________________________________________________________________


In [25]:
inf_dec_h_in = Input(shape=(latent_dim,), name="decoder_h_in")
inf_dec_c_in = Input(shape=(latent_dim,), name="decoder_c_in")

inf_dec_lstm_out, inf_dec_h_out, inf_dec_c_out = decoder_lstm(
    decoder_in, initial_state=[inf_dec_h_in, inf_dec_c_in])

inf_dec_out = decoder_dense(inf_dec_lstm_out)

inf_decoder = Model(
    [decoder_in, inf_dec_h_in, inf_dec_c_in],
    [inf_dec_out, inf_dec_h_out, inf_dec_c_out])

In [26]:
inf_decoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_in (InputLayer)        [(None, None, 87)]   0           []                               
                                                                                                  
 decoder_h_in (InputLayer)      [(None, 256)]        0           []                               
                                                                                                  
 decoder_c_in (InputLayer)      [(None, 256)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 256),  352256      ['decoder_in[0][0]',             
                                 (None, 256),                     'decoder_h_in[0][0]',     

Test trained model on the first 100 samples from both the training and validation sets.

In [27]:
max_out_seq_len = max([len(seq) for _, seq in samples])
print("Max output length: ", max_out_seq_len)

start_token_idx = out_token2int[start_token]
stop_token_idx = out_token2int[stop_token]

Max output length:  87


In [28]:
def translate_sequence(one_hot_seq, encoder, decoder):
    encoding = encoder.predict(one_hot_seq)

    decoder_in = np.zeros((1, 1, out_vocab_size), dtype=np.float32)

    translated_text = ''
    done_decoding = False
    decoded_idx = start_token_idx
    while not done_decoding:
        decoder_in[0, 0, decoded_idx] = 1
        decoding, h, c = decoder.predict([decoder_in] + encoding)
        encoding = [h, c]
        decoder_in[0, 0, decoded_idx] = 0

        decoded_idx = np.argmax(decoding[0, -1, :])
        
        if decoded_idx == stop_token_idx:
            done_decoding = True
        else:
            translated_text += out_int2token[decoded_idx]

        if len(translated_text) >= max_out_seq_len:
            done_decoding = True
            
    return translated_text

Validation samples:

In [29]:
from seq2seq_util import test_predictions

test_predictions(valid_samples[:100], inf_encoder, inf_decoder, encode_batch, translate_sequence)

-----------------------------------------
Input sentence: A todos nos gusta montar en bici.
Dataset translation: 	We all like cycling.

Model output: All the some are something.
-----------------------------------------
Input sentence: Tom se rió de todos los chistes de Mary.
Dataset translation: 	Tom laughed at all of Mary's jokes.

Model output: Tom stopped Mary a little the back.
-----------------------------------------
Input sentence: Tom es un asqueroso.
Dataset translation: 	Tom is a creep.

Model output: Tom is a good the back.
-----------------------------------------
Input sentence: ¿Cuál es tu meta en la vida?
Dataset translation: 	What's your aim in life?

Model output: What's your favorite the back?
-----------------------------------------
Input sentence: Ella le escucha, aunque nadie más lo haga.
Dataset translation: 	She listens to him even though no one else does.

Model output: She is almost the start the book to do.
-----------------------------------------
Input sen

Training samples:

In [30]:
test_predictions(train_samples[:100], inf_encoder, inf_decoder, encode_batch, translate_sequence)

-----------------------------------------
Input sentence: Después de una larga espera pudimos entrar.
Dataset translation: 	We got in after a long wait.

Model output: An the start is a lot of the back.
-----------------------------------------
Input sentence: Lo siento, pero es imposible.
Dataset translation: 	I'm sorry, but it's impossible.

Model output: I'm sorry, I was to stop.
-----------------------------------------
Input sentence: Parecía satisfecho.
Dataset translation: 	He looked pleased.

Model output: It seems to stop.
-----------------------------------------
Input sentence: Saqué el pastel del horno.
Dataset translation: 	I took the cake out of the oven.

Model output: I was a little to the back.
-----------------------------------------
Input sentence: Es un trabajo muy difícil.
Dataset translation: 	That's a very tough job.

Model output: It's a stranger to the back.
-----------------------------------------
Input sentence: Dijiste que no entendías.
Dataset translation

Export model in Core ML format.

In [31]:
coreml_enc_in = Input(shape=(None, in_vocab_size), name="encoder_in")
coreml_enc_lstm = LSTM(latent_dim, return_state=True, name="encoder_lstm")
coreml_enc_out, _, _ = coreml_enc_lstm(coreml_enc_in)

coreml_encoder_model = Model(coreml_enc_in, coreml_enc_out)
coreml_encoder_model.output_layers = coreml_encoder_model._output_layers

inf_encoder.save_weights("Es2EnCharEncoderWeights.h5")
coreml_encoder_model.load_weights("Es2EnCharEncoderWeights.h5")

**后面的内容因为训练时的所用环境问题没法及时运行，我也不行改动太多原notebook代码，移步`convert_model_myself.ipynb`**

In [45]:
import coremltools

coreml_encoder = coremltools.converters.convert(
    coreml_encoder_model,
    input_names="encodedSeq",
    output_names="ignored")

coreml_encoder.save("Es2EnCharEncoder.mlmodel")

<class 'keras.engine.functional.Functional'>


TypeError: convert() got an unexpected keyword argument 'input_names'

In [44]:
coreml_dec_in = Input(shape=(None, out_vocab_size))

coreml_dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
coreml_dec_lstm_out, _, _ = coreml_dec_lstm(coreml_dec_in)
coreml_dec_dense = Dense(out_vocab_size, activation="softmax")
coreml_dec_out = coreml_dec_dense(coreml_dec_lstm_out)

coreml_decoder_model = Model(coreml_dec_in, coreml_dec_out)
coreml_decoder_model.output_layers = coreml_decoder_model._output_layers

inf_decoder.save_weights("Es2EnCharDecoderWeights.h5")
coreml_decoder_model.load_weights("Es2EnCharDecoderWeights.h5")

In [27]:
coreml_decoder = coremltools.converters.convert(
    coreml_decoder_model,
    input_names="encodedChar",
    output_names="nextCharProbs")

coreml_decoder.save("Es2EnCharDecoder.mlmodel")

2021-12-17 16:30:14.829675: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-12-17 16:30:14.842193: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 113 nodes (60), 133 edges (69), time = 2.493ms.
  function_optimizer: Graph size after: 113 nodes (0), 133 edges (0), time = 1.399ms.
Optimization results for grappler item: while_body_3422
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
Optimization results for grappler item: while_cond_3421
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-12-17 16:30:14.935392: I tensorflow/core/grappler/devices.cc:75] Number of e

Convert weights to 16bit floats. This shouldn't hurt performance much, if at all, and it reduces the app's download size.

In [40]:
from coremltools.models.neural_network import quantization_utils

def convert_to_fp16(mlmodel_filename):
    model_fp32 = coremltools.models.MLModel(mlmodel_filename)
    spec_16bit = quantization_utils.quantize_weights(model_fp32, nbits=16)
    coremltools.utils.save(spec_16bit, f"{mlmodel_filename}16Bit.mlmodel")

In [ ]:
convert_to_fp16("Es2EnCharEncoder.mlmodel")
convert_to_fp16("Es2EnCharDecoder.mlmodel")

Save the maps so you can transform text to and from ints. You'll need them later in the iOS app.

In [ ]:
import json

with open("esCharToInt.json", "w") as f:
    json.dump(in_token2int, f)
with open("intToEnChar.json", "w") as f:
    json.dump(out_int2token, f)